# 03. 결과 분석 및 시각화

모델 예측 결과를 분석하고 최종 입지 추천을 시각화합니다.

## 주요 내용
- 상위/하위 지역 비교
- 피처별 영향도 분석
- 최종 입지 추천

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('..')

from src.data_loader import load_csv
from src.visualization import ResultVisualizer, set_korean_font

# 한글 폰트 설정 (OS 자동 감지)
set_korean_font()

%matplotlib inline

## 1. 결과 데이터 로드

In [ ]:
# 예측 결과 로드
results_df = load_csv('../data/processed/final_analysis_data.csv')
results_df = results_df.sort_values('value', ascending=False)

print(f'총 {len(results_df)}개 행정동')
results_df.head(10)

## 2. 상위/하위 지역 분석

In [ ]:
# 상위/하위 15개 지역
top15 = results_df.head(15)
bottom15 = results_df.tail(15)

print('=== 상위 15개 지역 ===')
if '행정동' in top15.columns:
    print(top15[['행정동', 'value']].to_string(index=False))
else:
    print(top15.head())

In [ ]:
# 상위/하위 비교 시각화
viz = ResultVisualizer(figsize=(12, 6))
viz.set_output_dir('../outputs/figures')

# 비교할 피처 선택 (숫자형 컬럼 중 선택)
numeric_cols = results_df.select_dtypes(include=[np.number]).columns.tolist()
compare_cols = [c for c in numeric_cols if c != 'value'][:8]

if compare_cols:
    viz.plot_district_comparison(
        top15, bottom15,
        columns=compare_cols,
        title='상위 vs 하위 15개 지역 피처 비교',
        save_name='district_comparison.png'
    )
    plt.show()

## 3. 예측 점수 분포

In [ ]:
# 예측 점수 분포
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 히스토그램
axes[0].hist(results_df['value'], bins=30, edgecolor='black', alpha=0.7)
axes[0].axvline(results_df['value'].mean(), color='red', linestyle='--', label=f'평균: {results_df["value"].mean():.2f}')
axes[0].set_title('예측 점수 분포')
axes[0].set_xlabel('예측 점수')
axes[0].set_ylabel('빈도')
axes[0].legend()

# 상위 15개 지역 바 차트
top15_sorted = top15.sort_values('value')
y_labels = top15_sorted['행정동'] if '행정동' in top15_sorted.columns else range(15)
axes[1].barh(range(15), top15_sorted['value'], color='steelblue')
axes[1].set_yticks(range(15))
axes[1].set_yticklabels(y_labels)
axes[1].set_title('상위 15개 추천 지역')
axes[1].set_xlabel('예측 점수')

plt.tight_layout()
plt.savefig('../outputs/figures/prediction_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. 결론

In [ ]:
print('='*50)
print('서울시 밀키트 매장 최적 입지 분석 결과')
print('='*50)
print()
print('1. 상위 추천 지역:')
if '행정동' in top15.columns:
    for i, row in top15.head(5).iterrows():
        print(f'   - {row["행정동"]} (점수: {row["value"]:.2f})')
print()
print('2. 주요 영향 피처:')
print('   - 30~50대 인구 비율')
print('   - 1~2인 가구 수')
print('   - 대중교통 접근성')
print('   - 맞벌이 가구 비율')
print()
print('3. 한계점:')
print('   - 실제 매출 데이터 없이 프록시 변수 사용')
print('   - 표본 크기 제한으로 일반화 주의 필요')